In [17]:
%cd ../..

C:\Users\qaz89\documents\Git Repos


In [20]:
!cd

C:\Users\qaz89\documents\Git Repos


In [1]:
import os
import sys
print(sys.version)

3.7.4 (tags/v3.7.4:e09359112e, Jul  8 2019, 20:34:20) [MSC v.1916 64 bit (AMD64)]


In [2]:
import numpy as np

import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.utils.model_zoo as model_zoo
import torchvision.transforms as transforms
import torchvision

import pdb
import math
import matplotlib.pyplot as plt
import glob
import PIL

### Build the model

In [24]:
# practice building the model
class SimpleCNN(nn.Module):
    
    def __init__(self, num_channels=1, num_classes=10):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(num_channels, 32, 3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 32, 3, stride=1, padding=1)
        self.pool1 = nn.MaxPool2d(2)
        self.drop1 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(14*14*32, 128)
        self.drop2 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, num_classes)
        
    def forward(self, X):
        X = F.relu(self.conv1(X))
        X = F.relu(self.conv2(X))
        X = self.pool1(X)
        X = self.drop1(X)
        X = X.reshape(-1, 14*14*32)
        X = F.relu(self.fc1(X))
        X = self.drop2(X)
        X = self.fc2(X)
        
        return X # logits

### Define helper function for decoding segmentation map
<img src="https://github.com/ykchua1/deepLrn/raw/master/images/fig_segmap.PNG" alt="fig" width="150"/>

Inspired by this [webpage](https://www.learnopencv.com/pytorch-for-beginners-semantic-segmentation-using-torchvision/)

In [25]:
def decode_segmap(image, nc=3):
    
    # define the class to color mapping
    label_colors = np.array([(0,0,0), (128,0,0), (0,128,0)])
    
    r = np.zeros_like(image).astype(np.uint8)
    g = np.zeros_like(image).astype(np.uint8)
    b = np.zeros_like(image).astype(np.uint8)
    
    for label in range(0, nc):
        idx = image == label
        r[idx] = label_colors[label, 0]
        g[idx] = label_colors[label, 1]
        b[idx] = label_colors[label, 2]
        
    rgb = np.stack([r, g, b], axis=2)
    return rgb    